In [1]:
# Install the Google AI Python SDK
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata
import os
import shutil
import time

# --- 1. CONFIGURE THE API KEY ---
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully!")
except Exception as e:
    print(f"🛑 Configuration failed. Error: {e}")

# --- 2. CREATE THE MESSY FOLDER AND FILES ---
if os.path.exists('my_messy_folder'):
    shutil.rmtree('my_messy_folder')
os.makedirs('my_messy_folder', exist_ok=True)
dummy_files = {
    "report_q3.txt": "This is the quarterly financial report for Q3.",
    "project_alpha.py": "import os\n\nprint('Hello from project alpha!')",
    "vacation_photo.jpg": "some_image_data_here",
    "notes_on_ai.txt": "Gemini is a large language model from Google.",
    "sales_data_2025.csv": "Date,Product,Revenue\n2025-08-01,Widget,500",
    "archive.zip": "some_zip_data",
    "app_styles.css": "body { font-family: 'Arial', sans-serif; }",
    "meeting_transcript_aug_1.txt": "Discussion points from the Friday meeting."
}
for filename, content in dummy_files.items():
    with open(os.path.join('my_messy_folder', filename), 'w') as f:
        f.write(content)
print("\n📂 Created a messy folder.")
print("="*40 + "\n")


# --- 3. DEFINE THE AI CLASSIFICATION FUNCTION (USING FLASH MODEL) ---
def get_file_category(file_path):
    """Uses Gemini to classify a file into a category, with a robust retry mechanism."""
    max_retries = 3
    retry_delay = 5

    for attempt in range(max_retries):
        try:
            with open(file_path, 'r', errors='ignore') as f:
                content_preview = f.read(5000)

            categories = ["Code", "Documents", "Images", "Data", "Archives", "Web", "Other"]
            prompt = f"""
            Analyze the file name and content, then classify it into one category: {', '.join(categories)}.
            Return only the single category name.
            File: "{os.path.basename(file_path)}"
            Content: "{content_preview}"
            Category:
            """

            # --- CHANGE 1: Using the faster 'flash' model ---
            model = genai.GenerativeModel('gemini-2.5-flash')
            response = model.generate_content(prompt)

            category = response.text.strip().replace('*', '')

            if category in categories:
                return category
            else:
                for valid_cat in categories:
                    if valid_cat in category:
                        return valid_cat
                return "Other"

        except Exception as e:
            print(f"   -> ❗️ Attempt {attempt + 1} failed for '{os.path.basename(file_path)}'.")
            if attempt < max_retries - 1:
                print(f"   -> ⏳ Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2
            else:
                print(f"   -> 🛑 All retries failed. Assigning 'Other'.")
                return "Other"
    return "Other"

# --- 4. RUN THE ORGANIZATION SCRIPT (WITH A DELAY) ---
source_dir = 'my_messy_folder'
print("🚀 Starting the organization process...\n")

files_to_organize = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

for filename in files_to_organize:
    file_path = os.path.join(source_dir, filename)
    category = get_file_category(file_path)
    print(f"File: '{filename}'  ->  🤖 Category: {category}")

    target_dir = os.path.join(source_dir, category)
    os.makedirs(target_dir, exist_ok=True)

    shutil.move(file_path, target_dir)

    time.sleep(1)

print("\n✨ Organization complete!")

✅ Gemini API configured successfully!

📂 Created a messy folder.

🚀 Starting the organization process...

File: 'sales_data_2025.csv'  ->  🤖 Category: Data
File: 'vacation_photo.jpg'  ->  🤖 Category: Images
   -> ❗️ Attempt 1 failed for 'report_q3.txt'.
   -> ⏳ Retrying in 5 seconds...
File: 'report_q3.txt'  ->  🤖 Category: Documents
File: 'archive.zip'  ->  🤖 Category: Archives
File: 'app_styles.css'  ->  🤖 Category: Code
File: 'notes_on_ai.txt'  ->  🤖 Category: Documents
File: 'meeting_transcript_aug_1.txt'  ->  🤖 Category: Documents
   -> ❗️ Attempt 1 failed for 'project_alpha.py'.
   -> ⏳ Retrying in 5 seconds...
File: 'project_alpha.py'  ->  🤖 Category: Code

✨ Organization complete!


In [2]:

!ls -R my_messy_folder


my_messy_folder:
Archives  Code	Data  Documents  Images

my_messy_folder/Archives:
archive.zip

my_messy_folder/Code:
app_styles.css	project_alpha.py

my_messy_folder/Data:
sales_data_2025.csv

my_messy_folder/Documents:
meeting_transcript_aug_1.txt  notes_on_ai.txt  report_q3.txt

my_messy_folder/Images:
vacation_photo.jpg
